In [1]:
import numpy as np
import cv2 as cv
import os
import json
import matplotlib.pyplot as plt
from PIL import Image
from lang_sam import LangSAM
from scipy.spatial.transform import Rotation as R
from lac.localization.ekf import EKF
import typing as T
from plotly import graph_objects as go
from lac.plotting import pose_trace, vector_trace

%load_ext autoreload
%autoreload 2

Given an image containing the lander, generate range and bearing measurements to the lander.


In [ ]:
# Define camera intrinsics
W, H = 1280, 720
FOV = 1.22  # radians
focal_length_x = W / (2 * np.tan(FOV / 2))
focal_length_y = H / (2 * np.tan(FOV / 2))
K = np.array([[focal_length_x / 2, 0, W / 2], [0, focal_length_y / 2, H / 2], [0, 0, 1]])

In [ ]:
## ---- TEST IMAGES FOR FIDUCIAL SHAPE SEG -------------------------------
# data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/data_collection_1")

# data_collection_1
# true examples
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/35.png # group D
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/124.png # group D
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/1527.png # group A
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/1620.png # group A

# false examples
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/328.png
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/619.png
# /home/lac/LunarAutonomyChallenge/output/data_collection_1/front_left/974.png

# lander_closeups
# true examples
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/53.png

# i = [35, 124, 328, 619, 974, 1527, 1620]
# ------------------------------------------------------------


# for ind in i:
#     I1 = cv.imread(os.path.join(data_path, "FrontLeft", f"{ind}.png"), cv.IMREAD_GRAYSCALE)
#     plt.imshow(I1, cmap="gray")
#     plt.show()

# Without fiducials

Use segmentation to identify the lander outline. Next, identify extreme points on the lander outline.
Assuming that we know the lander geometry, we know the location of these points in 3D relative to the
lander center.

Alternatively, we can try to directly predict the lander center (in the image in image coordinates)
from the segmentation.


In [ ]:
data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/lander_no_fiducials_left_far/")
CAM_NAME = "Left"
imgs = os.listdir(os.path.join(data_path, CAM_NAME))
imgs = [os.path.join(data_path, CAM_NAME, imgs[i]) for i in range(0, len(imgs), 1)]
imgs.sort()

# # Segment lander
# model = LangSAM()
# text_prompt = "lander."
# SEG_SCORE_THRESH = 0.4 # requires more testing

# for img in imgs:
#     I1 = cv.imread(os.path.join(data_path, CAM_NAME, img), cv.IMREAD_GRAYSCALE)
#     plt.imshow(I1, cmap="gray")
#     plt.show()

#     image_seg_in = Image.open(os.path.join(data_path, CAM_NAME, img)).convert("RGB")
#     results = model.predict([image_seg_in], [text_prompt])
#     full_mask = np.zeros_like(image_seg_in).copy()
#     if len(results) == 0:
#         print("No masks found")
#     for i, mask in enumerate(results[0]["masks"]):
#         print(results[0]["scores"][i])
#         if results[0]["scores"][i] < SEG_SCORE_THRESH:
#             continue
#         full_mask[mask.astype(bool)] = 255

#         # convert mask to pixel coordinates to get min area rect
#         mask = mask.astype(np.uint8)
#         contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
#         if contours:
#             largest_contour = max(contours, key=cv.contourArea)  # Pick the biggest object
#             rect = cv.minAreaRect(largest_contour)
#             box = cv.boxPoints(rect)
#             box = np.intp(box)
#             cv.drawContours(full_mask, [box], 0, (0, 0, 255), 2)

#             # sort the box points by the y coordinate
#             box = box[np.argsort(box[:, 1])]
#             lower_corners = box[2:]
#             mid_point = np.mean(lower_corners, axis=0)
#             cv.circle(full_mask, tuple(mid_point.astype(int)), 10, (0, 0, 255), -1)

#         # ------ PRIOR ATTEMPTS TO GET CENTER POINT BETWEEN LANDER FEET ------
#         # # get bounding box
#         # x, y, w, h = cv.boundingRect(mask)
#         # cv.rectangle(full_mask, (x, y), (x+w, y+h), (0, 255, 0), 2)

#         # x_avg = int(x + w/2)
#         # y_max = int(y + h)
#         # cv.circle(full_mask, (x_avg, y_max), 10, (0, 255, 0), -1)

#         # # get average x and max y of the mask to get the center point
#         # mask_x_avg = int(np.mean(np.where(mask)[1]))
#         # mask_y_max = int(np.max(np.where(mask)[0]))
#         # cv.circle(full_mask, (mask_x_avg, mask_y_max), 10, (255, 0, 0), -1)

#     plt.imshow(full_mask, cmap="gray")
#     plt.show()


In [ ]:
def get_roteuler_tvec_from_poselist(poses: T.Union[T.List[np.ndarray], np.ndarray]) -> tuple:
    """
    Get Euler angles and translation vectors from poses. 4x4 for a single pose, Nx4x4 for a list of poses.
    Input:
        poses - list of poses or a single pose
    Output:
        Returns a tuple of Euler angles and tvec (lists if input is a list of poses)
    """

    def pose_to_roteuler_tvec(pose: np.ndarray) -> tuple:
        """Get rpy and tvec from a single pose"""
        r = R.from_matrix(pose[:3, :3])
        rpy = r.as_euler("xyz", degrees=False)
        tvec = pose[:3, 3]
        return rpy, tvec

    if len(poses.shape) == 3 or isinstance(poses, list):  # list of poses
        rpy, tvec = [], []
        for pose in poses:
            r, t = pose_to_roteuler_tvec(pose)
            rpy.append(r)
            tvec.append(t)
        return rpy, tvec

    rpy, tvec = pose_to_roteuler_tvec(poses)
    return rpy, tvec


class MeasModelLanderSeg:
    def __init__(
        self,
        model,
        lander_pose: np.ndarray,
        cam_pose: np.ndarray,
        seg_thresh: float = 0.4,
        K: np.ndarray = K,
        R_nom: np.ndarray = np.eye(3) * 0.01,
    ):
        self.model = model
        self.seg_thresh = seg_thresh
        self.cam_rpy_rover, self.cam_pos_rover = get_roteuler_tvec_from_poselist(
            cam_pose
        )  # camera position relative to rover in rover frame
        self.lander_rpy_world, self.lander_pos_world = get_roteuler_tvec_from_poselist(
            lander_pose
        )  # w.r.t. world frame
        self.K = K  # camera intrinsics
        self.R = R_nom  # measurement noise covariance matrix under nominal conditions

    def get_lander_origin_pix_from_image(self, img_path: str, display: bool = False) -> np.ndarray:
        """Measurement function for extracting the lander origin pixel coordinates from an image"""

        lander_origin = None

        # Segment lander
        image_seg_in = Image.open(img_path).convert("RGB")
        text_prompt = "lander."
        results = self.model.predict([image_seg_in], [text_prompt])
        full_mask = np.zeros_like(image_seg_in).copy()

        if len(results) == 0:  # no masks found
            return lander_origin

        for i, mask in enumerate(results[0]["masks"]):
            if results[0]["scores"][i] < self.seg_thresh:
                continue

            full_mask[mask.astype(bool)] = 255
            mask = mask.astype(np.uint8)
            contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

            if contours:
                largest_contour = max(contours, key=cv.contourArea)
                rect = cv.minAreaRect(largest_contour)
                box = cv.boxPoints(rect)
                box = np.intp(box)

                # sort the box points by the y coordinate to get the lower corners (lander feet)
                sorted_box = box[np.argsort(box[:, 1])]
                lower_corners = sorted_box[2:]
                mid_point = np.mean(lower_corners, axis=0)
                lander_origin = mid_point

                if display:
                    cv.drawContours(full_mask, [box], 0, (0, 0, 255), 2)
                    cv.circle(full_mask, tuple(mid_point.astype(int)), 10, (0, 0, 255), -1)

        if display:
            plt.imshow(full_mask, cmap="gray")
            plt.show()

        return lander_origin

    def get_lander_origin_pix_from_proj(
        self, rover_state: np.ndarray, display: bool = False
    ) -> np.ndarray:
        """
        Compute the lander origin pixel coordinates via projection to the image plane.
        NOTE: X_rot_Y is the rotation matrix that rotates the Y frame to the X frame.
        NOTE: rpy is roll, pitch, yaw in radians.

        Input:
            rover_state - state of the rover w.r.t. the world frame
        Output:
            lander_origin_pix - lander origin pixel coordinates [u, v]
        """

        OPENCVCAM_ROT_ROVER = np.array([[0, -1, 0], [0, 0, -1], [1, 0, 0]])

        rover_pos_world = rover_state[:3]
        rover_rpy_world = rover_state[6:]

        rover_rot_world = R.from_euler("xyz", rover_rpy_world, degrees=False).as_matrix()
        cam_rot_rover = R.from_euler("xyz", self.cam_rpy_rover, degrees=False).as_matrix()
        cam_rot_world = cam_rot_rover @ rover_rot_world
        opencvcam_rot_cam = OPENCVCAM_ROT_ROVER @ cam_rot_rover.T

        # print("RPY world to rover: ", R.from_matrix(rover_rot_world).as_euler("xyz", degrees=True))
        # print("RPY rover to cam: ", R.from_matrix(cam_rot_rover).as_euler("xyz", degrees=True))
        # print("RPY world to cam: ", R.from_matrix(cam_rot_world).as_euler("xyz", degrees=True))

        cam_pos_rover_world = rover_rot_world.T @ self.cam_pos_rover
        cam_pos_world = cam_pos_rover_world + rover_pos_world
        lander_pos_cam = opencvcam_rot_cam @ cam_rot_world @ (self.lander_pos_world - cam_pos_world)
        pix_coords = self.K @ lander_pos_cam

        if pix_coords[2] > 0:  # Check if the point is in front of the camera
            lander_origin_pix = pix_coords[:2] / pix_coords[2]
        else:
            print("Lander is behind the camera.")

        if display:
            # # plot the rover and camera axes in the rover frame
            # fig_rover = go.Figure()
            # rover_ref_frame = pose_trace((np.eye(3), np.zeros(3)))
            # cam_traces_rover = pose_trace((cam_rot_rover, self.cam_pos_rover))
            # cam_pos_rover_vectrace = vector_trace(np.zeros(3), self.cam_pos_rover, color="orange", name="cam_pos_rover")

            # for trace in rover_ref_frame:
            #     fig_rover.add_trace(trace)
            # for trace in cam_traces_rover:
            #     fig_rover.add_trace(trace)
            # for trace in cam_pos_rover_vectrace:
            #     fig_rover.add_trace(trace)
            # fig_rover.update_layout(height=700, width=1200, scene_aspectmode="data")
            # fig_rover.show()

            # plot the rover and camera axes in the world frame
            fig_world = go.Figure()

            lander_traces = pose_trace(
                (
                    R.from_euler("xyz", self.lander_rpy_world, degrees=False).as_matrix(),
                    self.lander_pos_world,
                )
            )
            rover_traces = pose_trace((rover_rot_world, rover_pos_world))
            cam_traces = pose_trace((cam_rot_world, cam_pos_world))

            opencv_traces = pose_trace((OPENCVCAM_ROT_ROVER @ rover_rot_world, cam_pos_world))

            rover_pos_world_vectrace = vector_trace(
                np.zeros(3),
                rover_pos_world,
                color="blue",
                name="[k] rover_pos w.r.t. world in world",
            )
            lander_pos_world_vectrace = vector_trace(
                np.zeros(3),
                self.lander_pos_world,
                color="purple",
                name="[k] lander_pos w.r.t. world in world",
            )

            cam_pos_world_vectrace = vector_trace(
                np.zeros(3), cam_pos_world, color="green", name="[c] cam_pos w.r.t. world in world"
            )

            cam_pos_rover_world_vectrace = vector_trace(
                rover_pos_world,
                rover_pos_world + cam_pos_rover_world,
                color="orange",
                name="[c] cam_pos w.r.t. rover in world",
                head_size=0.1,
            )

            los_vec_world = self.lander_pos_world - cam_pos_world
            los_vec_world_vectrace = vector_trace(
                cam_pos_world,
                cam_pos_world + los_vec_world,
                color="red",
                name="[c] LOS Cam to Lander in world",
            )

            for trace in lander_traces:
                fig_world.add_trace(trace)
            for trace in rover_traces:
                fig_world.add_trace(trace)
            for trace in cam_traces:
                fig_world.add_trace(trace)
            for trace in opencv_traces:
                fig_world.add_trace(trace)
            for trace in lander_pos_world_vectrace:
                fig_world.add_trace(trace)
            for trace in rover_pos_world_vectrace:
                fig_world.add_trace(trace)
            for trace in cam_pos_world_vectrace:
                fig_world.add_trace(trace)
            for trace in cam_pos_rover_world_vectrace:
                fig_world.add_trace(trace)
            for trace in los_vec_world_vectrace:
                fig_world.add_trace(trace)

            fig_world.update_layout(height=700, width=1200, scene_aspectmode="data")
            fig_world.show()

        return lander_origin_pix[0:2]

    def meas(self, rover_state: np.ndarray) -> np.ndarray:
        """
        Measurement function for the EKF.
        Input:
            state - state vector of the EKF
        Output:
            z - measurement vector
            H - measurement matrix
            R - measurement noise covariance matrix
        """

        # get the lander origin pixel coordinates via projection to the image plane
        lander_orig_pix = self.get_lander_origin_pix_from_proj(rover_state)

        # get the Jacobian
        H = np.zeros((2, len(rover_state)))

        # get the measurement noise covariance matrix
        # TODO: tune based on segmentation results? like if not the entire lander is visible in the image etc.

        return lander_orig_pix

In [ ]:
# Ground truth trajectory data loading
traj_data = json.load(open(f"{data_path}/data_log.json"))
initial_pose = np.array(traj_data["initial_pose"])
data_log_path = os.path.join(data_path, "data_log.json")
data_log = json.load(open(data_log_path))

# Lander known pose w.r.t. world frame
lander_pose_world = np.array(data_log["lander_pose_world"])

# Camera (front left) known pose w.r.t. rover body-fixed frame
CAM_POS_KEYS = ["x", "y", "z"]
CAM_ANG_KEYS = ["roll", "pitch", "yaw"]
cams_geoms = json.load(open(os.path.expanduser("~/LunarAutonomyChallenge/docs/geometry.json")))[
    "rover"
]["cameras"]
cam = cams_geoms[CAM_NAME.lower()]
rover_t_cam = np.array([cam[key] for key in CAM_POS_KEYS])
rover_r_cam = R.from_euler("xyz", [cam[key] for key in CAM_ANG_KEYS], degrees=False).as_matrix()
cam_pose = np.eye(4)
cam_pose[:3, :3] = rover_r_cam
cam_pose[:3, 3] = rover_t_cam

# Rover trajectory w.r.t. world frame
ROVER_TRAJ_KEYS = ["timestamp", "pose"]
frame_data = data_log["frames"]
gt_rover_traj_data = {key: [] for key in ROVER_TRAJ_KEYS}
for frame in frame_data:
    gt_rover_traj_data["timestamp"].append(frame["timestamp"])
    gt_rover_traj_data["pose"].append(frame["pose"])
gt_rover_traj_data["pose"] = np.array(gt_rover_traj_data["pose"])
gt_rover_roteulers, gt_rover_tvecs = get_roteuler_tvec_from_poselist(gt_rover_traj_data["pose"])
dt = np.diff(gt_rover_traj_data["timestamp"])

# # visualize data
# plt.figure(figsize=(10, 10))
# plt.plot(lander_pose_world[0, 3], lander_pose_world[1, 3], "ro", label="lander")
# plt.plot(gt_rover_traj_data["pose"][:, 0, 3], gt_rover_traj_data["pose"][:, 1, 3], "b-", label="rover")
# plt.show()

In [ ]:
# extract IMU measurements
imu_meas = []  # list of 6D IMU measurements (linear acceleration, angular velocity)
for frame in frame_data:
    imu_meas.append(np.array(frame["imu"], dtype=np.float64))

# extract camera measurements
lander_orig_pix_meas = []  # list of 2D pixel measurements of the lander origin
CAM_NAME = "Left"
model = LangSAM()
SEG_SCORE_THRESH = 0.4  # requires more testing
img_folder = os.path.join(data_path, CAM_NAME)

lander_orig_pix_meas = []
meas_model = MeasModelLanderSeg(
    model, lander_pose_world, cam_pose, seg_thresh=SEG_SCORE_THRESH, K=K
)

for frame in frame_data[939:942]:
    frame_id = frame["frame"]
    print(f"Processing frame {frame_id}")
    img_name = os.path.join(img_folder, f"{frame_id}.png")

    if os.path.exists(img_name):
        lander_orig_pix_meas.append(
            meas_model.get_lander_origin_pix_from_image(img_name, display=True)
        )
        print("True Pixels: ", lander_orig_pix_meas[-1])

        # test reproj
        rover_state = np.array(frame["pose"])
        rover_rpy, rover_tvec = get_roteuler_tvec_from_poselist(rover_state)
        rover_state = np.concatenate([rover_tvec, np.zeros(3), rover_rpy])
        print(
            "Reproj Pixels: ", meas_model.get_lander_origin_pix_from_proj(rover_state, display=True)
        )


In [ ]:
# EKF
v0 = np.zeros(3)
x0 = np.hstack((gt_rover_tvecs[0][:3], v0, gt_rover_roteulers[0])).T

init_r = 0.001
init_v = 0.01
init_angle = 0.001
P0 = np.diag(
    np.hstack(
        (
            np.ones(3) * init_r * init_r,
            np.ones(3) * init_v * init_v,
            np.ones(3) * init_angle * init_angle,
        )
    )
)

# Process noise
Q_r = 0.00**2
Q_v = 0.00**2
Q_angle = 0.00005**2
Q = np.diag(np.hstack((np.ones(3) * Q_r, np.ones(3) * Q_v, np.ones(3) * Q_angle)))

# Measurement noise
R = np.diag(np.array([1, 1]))  # 2D pixel measurements

x_store = np.zeros((len(gt_rover_tvecs), 9))
P_store = np.zeros((len(gt_rover_tvecs), 9, 9))

ekf = EKF(x0, P0)


# Using fiducials


## Segementation


In [2]:
data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/lander_closeups/")
# ----- lander closeups -----------------------------------------------
# true examples
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/31.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/65.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/97.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/147.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/181.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/265.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/353.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/369.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/411.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/457.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/493.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/545.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/1199.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/1269.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/1665.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/1705.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/1751.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/2043.png

# difficult examples
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/799.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/867.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/1111.png

# false examples
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/617.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/709.png
# /home/lac/LunarAutonomyChallenge/output/lander_closeups/FrontLeft/763.png

i = [
    31,
    65,
    97,
    147,
    181,
    265,
    353,
    369,
    411,
    457,
    493,
    545,
    1199,
    1269,
    1665,
    1705,
    1751,
    2043,
    799,
    867,
    1111,
    617,
    709,
    763,
]

In [ ]:
# Segment fiducials
model = LangSAM()
text_prompt = "square."
SEG_SCORE_THRESH = 0.4  # requires more testing

# for ind in i:
for ind in i:
    I1 = cv.imread(os.path.join(data_path, "FrontLeft", f"{ind}.png"), cv.IMREAD_GRAYSCALE)
    plt.imshow(I1, cmap="gray")
    plt.show()

    image_seg_in = Image.open(os.path.join(data_path, "FrontLeft", f"{ind}.png")).convert("RGB")
    results = model.predict([image_seg_in], [text_prompt])
    full_mask = np.zeros_like(image_seg_in).copy()
    for i, mask in enumerate(results[0]["masks"]):
        print(results[0]["scores"][i])
        if results[0]["scores"][i] < SEG_SCORE_THRESH:
            continue
        full_mask[mask.astype(bool)] = 255

    plt.imshow(full_mask, cmap="gray")
    plt.show()

In [ ]:
# # Ground truth trajectory data loading
# traj_data = json.load(open(f"{data_path}/data_log.json"))
# initial_pose = np.array(traj_data["initial_pose"])
# data_log_path = os.path.join(data_path, "data_log.json")
# data_log = json.load(open(data_log_path))

# # Lander known pose w.r.t. world frame
# lander_pose_world = np.array(data_log["lander_pose_world"])

# # Rover trajectory w.r.t. world frame
# rover_traj_keys = ["timestamp", "pose"]
# frame_data = data_log["frames"]
# rover_traj_data = {key: [] for key in rover_traj_keys}
# for frame in frame_data:
#     rover_traj_data["timestamp"].append(frame["timestamp"])
#     rover_traj_data["pose"].append(frame["pose"])
# rover_traj_data["pose"] = np.array(rover_traj_data["pose"])

# # Camera (front left) known pose w.r.t. rover body-fixed frame
# CAM_KEY = "front left"
# CAM_POS_KEYS = ["x", "y", "z"]
# CAM_ANG_KEYS = ["roll", "pitch", "yaw"]
# camera_geom = json.load(open(os.path.expanduser("~/LunarAutonomyChallenge/docs/geometry.json")))["rover"]["cameras"]
# cam_front_left = camera_geom[CAM_KEY]
# cam_front_left_pos = np.array([cam_front_left[key] for key in CAM_POS_KEYS])
# rover_r_camfl = R.from_euler("xyz", [cam_front_left[key] for key in CAM_ANG_KEYS], degrees=False).as_matrix()

# # known fiducial corner positions
# fiducial_group_centers = json.load(open(os.path.expanduser("~/LunarAutonomyChallenge/docs/geometry.json")))["lander"]["fiducials"]
# centers_group_a = fiducial_group_centers["a"]
# centers_group_b = fiducial_group_centers["b"]
# centers_group_c = fiducial_group_centers["c"]
# centers_group_d = fiducial_group_centers["d"]


In [ ]:
# # use frame 35 as the initial frame -- first-ish frame with apriltags visible
# init_frame = 35
# # rover_init_pose  = rover_traj_data["pose"][0, :, :]
# rover_init_pose = rover_traj_data["pose"][init_frame, :, :]
# I1 = cv.imread(os.path.join(data_path, "FrontLeft", f"{init_frame}.png"), cv.IMREAD_GRAYSCALE)
# plt.imshow(I1, cmap="gray")
# plt.show()

In [ ]:
# # extract the fiducial shapes from the initial frame
# model = LangSAM()
# text_prompt = "square."
# image_seg_in = Image.open(os.path.join(data_path, "FrontLeft", f"{init_frame}.png")).convert("RGB")
# results = model.predict([image_seg_in], [text_prompt])
# full_mask = np.zeros_like(image_seg_in).copy()
# for i, mask in enumerate(results[0]["masks"]):
#     print(results[0]["scores"][i])
#     if results[0]["scores"][i] < 0.45:
#         # TODO: the score threshold will require tuning
#         continue
#     full_mask[mask.astype(bool)] = 255

# plt.imshow(full_mask, cmap="gray")
# plt.show()

# # extract the center of each mask
# mask_centers = []
# for i, mask in enumerate(results[0]["masks"]):
#     if results[0]["scores"][i] < 0.45:
#         continue
#     mask = mask.astype(bool)
#     mask_center = np.mean(np.argwhere(mask), axis=0)
#     mask_centers.append(mask_center)

# # plot on image
# plt.imshow(I1, cmap="gray")
# for center in mask_centers:
#     plt.plot(center[1], center[0], "ro", markersize=2)
# plt.show()

In [ ]:
# # sort mask centers by y-coordinate
# mask_centers = np.array(mask_centers)
# mask_centers = mask_centers[np.argsort(mask_centers[:, 0])]

# # sort into top-left top-right bottom-right bottom-left
# top_centers = mask_centers[:2]
# bottom_centers = mask_centers[2:]
# top_centers = top_centers[np.argsort(top_centers[:, 1])]
# bottom_centers = bottom_centers[np.argsort(bottom_centers[:, 1])]
# bottom_centers = bottom_centers[::-1]
# mask_centers = np.vstack([top_centers, bottom_centers])

# # reverse the column order to match the fiducial group centers
# mask_centers = mask_centers[:, ::-1]

In [ ]:
# # solve PnP
# # TODO: get which fidicucial group is in the photo via rover pose estimate
# TAG_KEYS = ["top left", "top right", "lower right", "lower left"]
# TAG_CENTER_KEYS = ["x", "y", "z"]
# world_pts = np.array([[centers_group_d[key_name][key] for key in TAG_CENTER_KEYS] for key_name in TAG_KEYS])
# image_pts = mask_centers

# print("World points:\n", world_pts)
# print("Image points:\n", image_pts)

# pnp_success, camfl_rvec_world, camfl_tvec_world = cv.solvePnP(world_pts, mask_centers, K, None)
# if pnp_success:
#     # extract rotation and translation
#     camfl_r_world, _ = cv.Rodrigues(camfl_rvec_world)
#     print("Rotation matrix:\n", camfl_r_world)
#     print("Translation vector:\n", camfl_tvec_world)

In [ ]:
# # from scipy.spatial.transform import Rotation as R
# rover_init_pose

## AprilTag


In [ ]:
data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/lander/")